In [ ]:
import cv2
import os
from datetime import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
import pymongo
from pymongo import MongoClient
from tensorflow.keras.models import model_from_json, load_model
from flask import Flask, render_template, Response, request

c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


### Load Models

In [ ]:
#load facedetector.h5 model
facedetector = load_model('Facedetector.h5')
# emotion = load_model('E:\PROJECT\Best accu\emotion_model.h5')

# load model from JSON file
emotion = model_from_json(open("fer.json", "r").read())  
# load weights into the new model
emotion.load_weights('fer_weights.h5') 

em ={"angry":0, "disgust":1, "fear":2, "happy":3, "neutral":4, "sad":5, "surprise":6}
op = dict(zip(em.values(), em.keys()))

### Connect to remote MongoDB server

In [ ]:
cluster = MongoClient('mongodb+srv://<username>:<password>@cluster0.hsgyw40.mongodb.net/?retryWrites=true&w=majority')
db = cluster['Face_Emotion']
collection = db['Emotion']

### Web Application

In [ ]:
app = Flask(__name__)
@app.route('/')
def index():
    return render_template('Index.html')

def gen_frames():
    df = pd.DataFrame({'Time':[],'Frame': [], 'Face': [],
                       'Emotion': [], '% of accuracy':[]})
    em ={"angry":0, "disgust":1, "fear":2, "happy":3, "neutral":4, "sad":5, "surprise":6}
    op = dict(zip(em.values(), em.keys()))

    cap = cv2.VideoCapture(0)
    i = 1
    
    while cap.isOpened():
        ret , frame = cap.read()
        if not ret:
            break
        else:
            frame = frame[50:500, 50:500, :]

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            resized = tf.image.resize(rgb, (120,120))
            resized -= np.mean(resized,0)
            resized /= np.std(resized,0)


            yhat = facedetector.predict(np.expand_dims(resized,0))
            sample_coords = yhat[1][0]
            x,y,w,h = np.multiply(sample_coords, [450,450,450,450]).astype(int)


            if yhat[0] > 0.5:
                cv2.rectangle(frame,
                             (x,y),
                             (w,h),
                             (255,0,0),2)


                crop = cv2.resize(gray[y:h, x:w], (48,48))
                crop = crop/255
                crop = np.array([crop])

                pred_value  = emotion.predict(crop)
                pred = np.argmax(pred_value)

                cv2.putText(frame, str(op[pred])+str (round(np.max(emotion.predict(crop)*100)))+'%',
                            tuple(np.add([x,y], [0, -5])),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                
                df=df.append({'Time':datetime.now(),'Frame': 'Frame_'+str(i), 
                      'Face': crop.tolist(),'Emotion': str(op[pred]), 
                      '% of accuracy':round(np.max(emotion.predict(crop)*100))}, ignore_index=True)
                i+=1
                
            cv2.waitKey(500)
            ret, buffer = cv2.imencode('.jpg', frame)
            
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
            
            df.reset_index(inplace=False)
            data_dict = df.to_dict("records")
            # Insert collection
            collection.insert_many(data_dict)

@app.route('/video_feed')
def video_feed():
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')


if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Aug/2022 21:50:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2022 21:50:05] "GET /css/bootstrap.min.css HTTP/1.1" 404 -
127.0.0.1 - - [28/Aug/2022 21:50:05] "GET /css/fontAwesome.css HTTP/1.1" 404 -
127.0.0.1 - - [28/Aug/2022 21:50:05] "GET /css/templatemo-main.css HTTP/1.1" 404 -
127.0.0.1 - - [28/Aug/2022 21:50:09] "GET /video_feed HTTP/1.1" 200 -


### fetch data from MongoDb

In [ ]:
y = collection.find()
pd.DataFrame(y)

,_id,Time,Frame,Face,Emotion,% of accuracy
0,62f213ed5e269352d1096676,2022-08-09 13:27:32.774,Frame_6,"[[[0.6588235294117647, 0.5294117647058824, 0.7...",happy,98.0
1,62f213ed5e269352d1096673,2022-08-09 13:27:28.209,Frame_3,"[[[0.7450980392156863, 0.6745098039215687, 0.4...",neutral,37.0
2,62f213ed5e269352d1096679,2022-08-09 13:27:37.132,Frame_9,"[[[0.7098039215686275, 0.6313725490196078, 0.4...",neutral,52.0
3,62f213ed5e269352d1096672,2022-08-09 13:27:26.579,Frame_2,"[[[0.8117647058823529, 0.788235294117647, 0.48...",sad,42.0
4,62f213ed5e269352d1096671,2022-08-09 13:27:25.049,Frame_1,"[[[0.023529411764705882, 0.11372549019607843, ...",sad,41.0
...,...,...,...,...,...,...
76,62f213ed5e269352d10966bf,2022-08-09 13:29:29.278,Frame_79,"[[[0.9058823529411765, 0.9686274509803922, 0.9...",sad,66.0
77,62f213ed5e269352d10966bc,2022-08-09 13:29:24.879,Frame_76,"[[[0.8862745098039215, 0.9372549019607843, 0.9...",happy,92.0
78,62f213ed5e269352d10966b9,2022-08-09 13:29:20.667,Frame_73,"[[[0.4, 0.7490196078431373, 0.7529411764705882...",happy,48.0
79,62f213ed5e269352d10966c0,2022-08-09 13:29:30.703,Frame_80,"[[[0.6745098039215687, 0.6549019607843137, 0.8...",happy,30.0
